# MNE INV

The purpose of this notebook is to see if MNE-Python will play nicely with our current inputs such that we can use it to compute the inverse solution.

In [1]:
from EMGinv_fns import *
from geometry_utilities import *
from plotter_utility_functions import *

import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd

import pyvistaqt
%matplotlib qt

In [2]:
NUM_ELECTRODES = 256
# ARM = 'left'
# NUM_ELECTRODES = 128
ARM = 'right'

# MIN_DIPOLE_DISTANCE = 7.0 #mm
MIN_DIPOLE_DISTANCE = 2.0 #mm

SAMPLE_RATE = 2000. # samples per second (from MUAPs file)

# MASK_FILE = f'Data/{EXPERIMENT}_muaps_mask.mat'
# TEMPLATES_FILE = f'Data/{EXPERIMENT}_muaps_template-{MUAP_ID:02d}.mat'
# COVARIANCE_FILE = f'Data/{EXPERIMENT}_muaps_covariance.mat'
ARM_DICOM_FILENAME = 'Data/R_Forearm.dcm'

ELECTRODES_FILE = f'Data/{NUM_ELECTRODES}simparm_electrode_pos.npy'
MNE_FWD_SOLUTIONS_FILE = f'Data/simp_arm_2mm-fwd.fif'
# EXTRACT_MUAPS_FROM_MASK = False;

In [45]:
# Loading source space and forward model generated in MNE-python, see Fwd_BEM_MNE.ipynb
# Load electrode positions
electrode_pos = np.load(ELECTRODES_FILE)

# Load forward model
mne_fwd = mne.read_forward_solution(MNE_FWD_SOLUTIONS_FILE)
fwd = mne_fwd['sol']['data']
pos = mne_fwd['source_rr'] 
xscaling, yscaling, zscaling = (MIN_DIPOLE_DISTANCE*1e-3, MIN_DIPOLE_DISTANCE*1e-3, MIN_DIPOLE_DISTANCE*1e-3) # Is the distance between dipoles in source space #np.repeat(np.abs(np.sum(np.diff(pos[0], axis = 0))),3)  

# Transformations specific to dataset",
# Need to remove some electrodes - get rid of the middle ones"
fwd = np.vstack((fwd[:64,:], fwd[192:,:]))
electrode_pos = np.vstack((electrode_pos[:64,:], electrode_pos[192:,:]))

# Removal of dipole sources - here remove sources lying within cylinders representing bone
# pos, fwd = bone_remover(pos, fwd, -9e-3, 0, 5e-3) #Ulnar
# pos, fwd = bone_remover(pos, fwd, 5e-3, -9e-3, 5e-3) # Radius

# Consider fwd model adjustments
# Condense the fwd such that there is only one dipole per voxel
dipole_ori = [0, 0, 1]
fwd = fwd_convertfixed(fwd, dipole_ori )

Reading forward solution from /Users/pokhims/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Documents/Coding/CMU_EMGSL/Data/simp_arm_2mm-fwd.fif...
    Reading a source space...
    [done]
    1 source spaces read
    Desired named matrix (kind = 3523 (FIFF_MNE_FORWARD_SOLUTION_GRAD)) not available
    Read EEG forward solution (23424 sources, 256 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame


In [4]:
# correct_chnames = mne_fwd.ch_names[:64] + ['Prox - AUX 1-1', 'Prox - AUX 1-2', 'Prox - AUX 1-3','Prox - TRIGGERS', 'Prox - STATUS', 'Prox - COUNTER',] +  mne_fwd.ch_names[-64:] + ['Dist - TRIGGERS', 'Dist - STATUS', 'Dist - COUNTER']
# correct_mapping = dict(zip(epochs.info.ch_names, correct_chnames))
# mne.rename_channels(epochs.info, correct_mapping, allow_duplicates=True)

In [11]:
def load_tmsitomne_combine(f_prox=None, f_dist=None, scale=1e-6):
    """ Combines the distal and proximal datasets from TMSI together into one MNE object.
    
    Parameters: 
    - f_prox (str): the path to the proximal file to be loaded.
    - f_dist (str): the path to the distal file to be loaded.
    - scale (float): the scaling factor.  By default it will convert the data from uV to V.

    Returns:
    - MNE_raw (MNE raw object): the combined MNE object.
    """
    if f_prox is None:
        f_prox = 'Data/Pok_2024_08_21_A_PROX_8.poly5'
    if f_dist is None:
        f_dist = 'Data/Pok_2024_08_21_B_DIST_8.poly5' 

    prox = load_tmsi_data(filename=f_prox)
    dist = load_tmsi_data(filename=f_dist)      
    # ch_names = ["Prox - " + s for s in prox[1]] + ["Dist - " + s for s in dist[1]]
    ch_names = mne_fwd.ch_names[:64] + ['Prox - AUX 1-1', 'Prox - AUX 1-2', 'Prox - AUX 1-3','Prox - TRIGGERS', 'Prox - STATUS', 'Prox - COUNTER',] +  mne_fwd.ch_names[-64:] + ['Dist - TRIGGERS', 'Dist - STATUS', 'Dist - COUNTER']
    fs = prox[2]
    ch_types = ['eeg']*64 + ['misc']*(len(prox[1])-64) + ['eeg']*64 + ['misc']*(len(dist[1])-64)

    num_channels = len(ch_names)
    dist_sec = np.where(dist[0][-3,:]==254)[0]
    prox_sec = np.where(prox[0][-3,:]==254)[0]

    dist_sample = dist[0][:, dist_sec[0]:dist_sec[-1]]
    prox_sample = prox[0][:, prox_sec[0]:prox_sec[-1]]

    # Scale the data channels
    dist_sample[:64] = dist_sample[:64]*scale
    prox_sample[:64] = prox_sample[:64]*scale

    del dist, prox
    # Create MNE raw object
    info = mne.create_info(ch_names, fs, ch_types)
    # The lengths of the two datasets are stil off, so remove some samples, still misaligned slightly
    MNE_raw = mne.io.RawArray(np.concatenate((prox_sample[:,1:-1], dist_sample), axis=0), info)

    return MNE_raw

In [12]:
# Load some data - to construct covariance matrices
MNE_raw = load_tmsitomne_combine(f_prox = 'Data/Pok_2024_08_21_A_PROX_8.poly5', f_dist='Data/Pok_2024_08_21_B_DIST_8.poly5', scale=1e-6)

Reading file  Data/Pok_2024_08_21_A_PROX_8.poly5
	 Number of samples:  892710 
	 Number of channels:  70 
	 Sample rate: 4000 Hz
Done reading data.
Creating RawArray with float64 data, n_channels=70, n_times=892710
    Range : 0 ... 892709 =      0.000 ...   223.177 secs
Ready.
Sample rate:  4000  Hz
Channel names:  ['UNI 01', 'UNI 02', 'UNI 03', 'UNI 04', 'UNI 05', 'UNI 06', 'UNI 07', 'UNI 08', 'UNI 09', 'UNI 10', 'UNI 11', 'UNI 12', 'UNI 13', 'UNI 14', 'UNI 15', 'UNI 16', 'UNI 17', 'UNI 18', 'UNI 19', 'UNI 20', 'UNI 21', 'UNI 22', 'UNI 23', 'UNI 24', 'UNI 25', 'UNI 26', 'UNI 27', 'UNI 28', 'UNI 29', 'UNI 30', 'UNI 31', 'UNI 32', 'UNI 33', 'UNI 34', 'UNI 35', 'UNI 36', 'UNI 37', 'UNI 38', 'UNI 39', 'UNI 40', 'UNI 41', 'UNI 42', 'UNI 43', 'UNI 44', 'UNI 45', 'UNI 46', 'UNI 47', 'UNI 48', 'UNI 49', 'UNI 50', 'UNI 51', 'UNI 52', 'UNI 53', 'UNI 54', 'UNI 55', 'UNI 56', 'UNI 57', 'UNI 58', 'UNI 59', 'UNI 60', 'UNI 61', 'UNI 62', 'UNI 63', 'UNI 64', 'AUX 1-1', 'AUX 1-2', 'AUX 1-3', 'TRIGGER

In [16]:
# Important to filter the data (highpass filter)
MNE_raw = MNE_raw.filter(l_freq=100, h_freq=None, )
# Estimate the noise covariance matrix on Epoched data.  This means that the noise covariance will be estimated on pre-stimulus periods
channel_data = MNE_raw['Prox - TRIGGERS'][0][0]-252
events = tmsi_eventextractor(channel_data)
event_dict = {'Ext': -2, 'Flex': -6} # Should be correct 
epochs = mne.Epochs(MNE_raw, events, event_dict, tmin=-4, tmax=4, baseline=None, preload=True)
# Consider setting an average EEG reference across each panel of 32. 
epochs.set_eeg_reference('average', projection=True)
epochs.apply_proj()
# epochs.plot(n_epochs=1, scalings='auto', );

# Set tmin and tmax based on experimental conditions.  In this case, the participant was not moving perfectly to the triggers.
noise_cov = mne.compute_covariance(epochs, method='auto', tmin=-2, tmax=0.01)
data_cov = mne.compute_covariance(epochs, method='auto', tmin=1, tmax=epochs.tmax)

noise_cov.plot(epochs.info,)
data_cov.plot(epochs.info,)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 100.00
- Lower transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 87.50 Hz)
- Filter length: 529 samples (0.132 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    1.1s


Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 20 events and 32001 original time points ...
0 bad epochs dropped
EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
    Created an SSP operator (subspace dimension = 1)
    Setting small EEG eigenvalues to zero (without PCA)
Reducing data rank from 128 -> 127
Estimating covariance using SHRUNK
Done.
Estimating covariance using DIAGONAL_FIXED
    EEG regularization : 0.1
Done.
Estimating covariance using EMPIRICAL
Done.
Using cross-validation to select the best estimator.
    EEG regularization : 0.1
    EEG regularization : 0.1
    EEG regularization : 0.1
Number of samples

(<Figure size 380x370 with 2 Axes>, <Figure size 380x370 with 1 Axes>)

In [17]:
epochs.info

<Info | 8 non-empty values
 bads: []
 ch_names: EMG 1-1, EMG 1-2, EMG 1-3, EMG 1-4, EMG 1-5, EMG 1-6, EMG 1-7, ...
 chs: 128 EEG, 9 misc
 custom_ref_applied: False
 highpass: 100.0 Hz
 lowpass: 2000.0 Hz
 meas_date: unspecified
 nchan: 137
 projs: Average EEG reference: on
 sfreq: 4000.0 Hz
>

In [18]:
noise_cov

<Covariance | kind : full, shape : (128, 128), range : [-5.1e-11, +2.3e-10], n_samples : 160819>

In [59]:
# forward operator with fixed source orientations
mne.convert_forward_solution(mne_fwd, surf_ori=True, force_fixed=True, copy=False)

# Solve the inverse equation to get the source time courses

# Region
snr = 3.0  # 3 is the default
inv_method = 'MNE'
lambda2 = 1.0 / snr ** 2

inverse_operator = mne.minimum_norm.make_inverse_operator(epochs.info, mne_fwd, noise_cov,
                                         loose=0, depth=None,  # previously loose = auto'
                                         fixed=True)

# stc_est_region = mne.minimum_norm.apply_inverse(epochs, inverse_operator, lambda2, inv_method, pick_ori=None)

# Select only epoch 5 to look at
stc_est_region = mne.minimum_norm.apply_inverse_epochs(epochs[5], inverse_operator, lambda2, inv_method, pick_ori=None)
stc_est_region = stc_est_region[0]  # This is unncessary if only looking at one epoch, but essentially, a list containing the stc_est for each of the epochs is produced.

    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Changing to fixed-orientation forward solution with surface-based source orientations...
    [done]
Computing inverse operator with 128 channels.
    128 out of 256 channels remain after picking
Selected 128 channels
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 1.6e-13 (2.2e-16 eps * 128 dim * 5.8  max singular value)
    Estimated rank (eeg): 127
    EEG: rank 127 computed from 128 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.
    largest singular value = 11.2674
    scaling factor to adjust the trace = 5.01387e+28 (nchan = 128 nzero = 1)
Preparing the inverse operator for use

In [60]:
# Threshold
thresh = 0.5
# Look at specific timepoint in the source activity - 20 for matlab template waveform, 30 for other one; ext_1 - 5s and 15s are interesting; flex_1 - 9s
t = 4

source_activity = stc_est_region.data[:, t]
waveform = epochs[5]._get_data(picks='eeg')[0]

# Reshape source activity to condense N source orientations into 1 per voxel - Confirmed works for 3 orientations, should work for more.
reshape_by = source_activity.shape[0] // pos.shape[0]
reshaped_act = np.array(source_activity.reshape((reshape_by, -1), order='F'))
source_activity = np.linalg.norm(reshaped_act, axis=0)

ind = np.abs(source_activity) > thresh*np.max(np.abs(source_activity))
source_activity = source_activity[ind]
pos_t = pos[ind]

# Plot the convex hull and the moved points
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# Plot electrode positions
ax.scatter(electrode_pos[:, 0], electrode_pos[:, 1], electrode_pos[:, 2], c=waveform[:,t], marker='o', alpha=0.2, cmap='turbo')
# Plot the source space
ax.scatter(pos_t[:, 0], pos_t[:, 1], pos_t[:, 2], c=source_activity, marker='s', alpha=0.8, cmap='viridis')
# Set labels
ax.set_xlabel('X Axis (m)')
ax.set_ylabel('Y Axis (m)')
ax.set_zlabel('Z Axis (m)')
n_channels, n_sources = fwd.shape
ax.set_title(f'{n_channels}-Channel {n_sources}-Dipole LCMV Beamformer')

# # Save the figure to a file in the created folder
# plt.savefig(f'{save_dir}/{EXPERIMENT}_3D-T{t}.png', dpi=300)

Text(0.5, 0.92, '128-Channel 23424-Dipole LCMV Beamformer')

Code runs, but it has not been checked.

Key improvements:
- Demonstrate Beamformer code (and ensure it lines up with previous implementation) - This provides confidence that the code is working as expected.
- Electrode referencing should be done per patch, not for all electrodes at once. 
- Look at result on a muap.  Right now, it's just a longer time series where the gesture was done multiple times.
- Bone removal